# Mały projekt 1: wizualizacja poziomu zanieczyszczeń (PM2.5)

Główny Inspektorat Ochrony Środowiska (GIOS) udostępnia dane o jakości powietrza w Polsce na stronie [https://powietrze.gios.gov.pl](https://powietrze.gios.gov.pl), tj. poziom stężenia pyłów PM2.5, PM10, SO2 i innych zanieczyszczeń. Dane te są szczególnie przydatne w analizach środowiskowych i zdrowotnych. W tym zadaniu interesują nas godzinne pomiary stężeń drobnego pyłu **PM2.5** (pyłu o średnicy poniżej 2.5 µm) w latach **2015, 2018, 2021 i 2024**. Pyły PM2.5 są one bardzo szkodliwe dla zdrowia, gdyż mogąc przenikać głęboko do układu oddechowego i krwiobiegu. Zadanie polega na przeprowadzeniu analizy danych wraz z opisami czynności oraz wykresami.

## Wymagania ogólne

- Rozwiązanie należy przygotować w formie **Jupyter Notebooka (`.ipynb`)**.  Rozwiązanie proszę przesłać przez Moodle’a.
- Wszystkie operacje czyszczenia i łączenia danych wykonaj **programistycznie**, bez ręcznej edycji plików.
- Do każdego punktu należy obowiązkowo dodać opis z analizą otrzymanych wyników.  
- Zadanie należy wykonać w wylosowanych parach. Obie osoby z pary przesyłają **identyczne pliki** i dodają na początku notebooka opis swojego wkładu.
- Ocena jest wspólna dla pary, a nie indywidualna.

## Dane wejściowe i metadane

- Każdy rok to archiwum ZIP dostępne pod adresem [https://powietrze.gios.gov.pl/pjp/archives/](https://powietrze.gios.gov.pl/pjp/archives/).
- Wewnątrz znajdują się pliki Excel, np. `2024_PM25_1g.xlsx`.
- Poniższa funkcja `download_gios_archive` pozwala odczytać odpowiedni zbior danych; resztę danych pomijamy.
- Użyj pliku metadanych (dostępnego na tej samej stronie), aby zaktualizować **kody stacji** (część stacji mogła w międzyczasie zmienić nazwę). W metadanych znajdziesz odpowiednie kolumny.

## Zadania

### 1. Wczytanie i czyszczenie danych

Wczytaj dane dla lat **2015, 2018, 2021 i 2024**, oczyścić je z niepotrzebnych wierszy oraz ujednolić ich format. Zaktualizuj stare kody stacji zgodnie z metadanymi. Pozostaw tylko stacje występujące we wszystkich czterech latach. Informacje o stacjach pomiarowych warto uzupełnić o miejscowości dostępne w metadanych, np. za pomocą MultiIndex: (miejscowość, kod stacji). Pomiary dokonane o północy (00:00:00) powinny być potraktowane jako dotyczące poprzedniego dnia. Połącz dane z trzech lat w jeden `DataFrame` i zapisz do pliku.

### 2. Średnie miesięczne + trend dla miast

Oblicz średnie miesięczne stężenie PM2.5 dla każdej stacji i roku. Dla **Warszawy** i **Katowic**, po uśrednieniu po wszystkich stacjach z tych miast, narysuj wykres liniowy pokazujący trend średnich miesięcznych wartości PM2.5 w 2015 i 2024 roku. Oś X - miesiące (1-12); oś Y - średnia wartość PM2.5; 4 linie trendu. Dołącz opis i interpretację obserwowanych różnic.

### 3. Heatmapa miesięcznych średnich

Dla każdej miejscowości przedstaw heatmapę średnich miesięcznych stężeń PM2.5 w latach 2015, 2018, 2021 i 2024 (oś X – miesiąc, oś Y – rok). Uśrednij wartości po wszystkich stacjach w danej miejscowości. Każdy panel (facet) ma odpowiadać jednej miejscowości. Dołącz interpretację obserwowanych wyników.

### 4. Dni z przekroczeniem normy (WHO)

Dla każdej stacji i roku policz liczbę dni, w których wystąpiło przekroczenie dobowej normy stężenia PM2.5, czyli 15 µg/m³ (źródło: [https://airscan.org/new-who-air-quality-guidelines-2021/](https://airscan.org/new-who-air-quality-guidelines-2021/)). Znajdź 3 stacje z najmniejszą i 3 stacje z największą liczbą dni z przekroczeniem normy dobowej w 2024 roku. Dla tych 6 stacji narysuj *grouped barplot*, gdzie oś X – stacje, oś Y – liczba dni z przekroczeniem, kolor – rok (2015, 2018, 2021, 2024). Dołącz opis i interpretację obserwowanych różnic.

## Dodatkowe wymagania i sugestie

- Notebook powinien zawierać *sanity checks*, np.:
  - liczba stacji w każdym pliku,
  - liczba dni w każdym roku,  
  - kilka przykładowych mapowań kodów stacji,  
- Wszystkie wykresy powinny mieć tytuły, legendy i krótki opis interpretacji.
- Zachęcamy do weryfikacji kodu napisanego przez drugą osobę, gdyż ocena jest wspólna.
- Można wykorzystać dowolne poznane biblioteki do analizy i wizualizacji danych w Pythonie.

## Kryteria oceny

- Zadanie 1: 3 pkt
- Zadanie 2: 2 pkt
- Zadanie 3: 1.5 pkt
- Zadanie 4: 2 pkt
- Jakość wyjaśnień, interpretacje, opis wkładu: 1.5 pkt



# Zad 1

In [ ]:
from configs import *
from read_process import *
from calculations import *
from visualizations import *

In [ ]:
dfs = {}
years = YEARS
for year in years:
    dataframe = download_gios_archive(year, GIOS_URL_IDS[year], GIOS_PM25_FILES[year])
    print(f"Processing for {year}")
    dfs[year] = process_raw_df(dataframe, year)
metadata = download_metadata(META_ID)

In [ ]:
filtered_metadata = process_metadata(metadata)
#bierzemy pod uwage te wiersze gdzie zaszła zmiana kodów
metadata_subset = filtered_metadata.dropna(subset=[OLD_STATION_NAME, NEW_STATION_NAME])
raw_mapper = dict(zip(metadata_subset[OLD_STATION_NAME],metadata_subset[NEW_STATION_NAME]))
code_mapper = atomize_dict(raw_mapper)

In [ ]:
for year, df in dfs.items():
    dfs[year] = update_codes(df, code_mapper)

In [ ]:
final_df = merge_years(dfs, metadata)

In [ ]:
final_df.to_csv("merged_DF.csv")

In [ ]:
final_df

# Zad 2
## Średnie miesięczne + trend dla miast
Oblicz średnie miesięczne stężenie PM2.5 dla każdej stacji i roku. Dla **Warszawy** i **Katowic**, po uśrednieniu po wszystkich stacjach z tych miast, narysuj wykres liniowy pokazujący trend średnich miesięcznych wartości PM2.5 w 2015 i 2024 roku. Oś X - miesiące (1-12); oś Y - średnia wartość PM2.5; 4 linie trendu. Dołącz opis i interpretację obserwowanych różnic.

In [ ]:
monthly_cities = group_monthly_cities(final_df)
monthly_cities.head()

In [ ]:
plot_city_trends(monthly_cities, EX2_YEARS, EX2_CITIES)

## Interpretacja zadania 2-ego
Po pierwsze możemy zauważyć, że w niektórych miesiącach poziomy zanieczyszczeń są o wiele większe niż innych np. w okresach zimowych średnie stężenie PM2.5 (najwyższy odczyt w lutym) jest zauważalnie większe niż w okresie letnim. To oczywiście spowodowane jest paleniem min. w piecach w gospodarstwach domowych.
Co ciekawsze w latach 2014 ogólny średni poziom zanieczysczeń jest większy niż w latach 2024 (linie trendu są wyżej), możemy, więc zauważyć efekty przepisów mających na celu ochronę środowiska, a także wymianę min. starych pieców grzewczych na nowe.
Ostatnią sprawą może być to, że średni poziom PM2.5 jest mniejszy w Warszawie niżeli w Katowicach (być może spowodowane tym, że na Śląsku w gospodarstwach domowych więcej było pieców węglowych).


# Zad 3
## Heatmapa miesięcznych średnich
Dla każdej miejscowości przedstaw heatmapę średnich miesięcznych stężeń PM2.5 w latach 2015, 2018, 2021 i 2024 (oś X – miesiąc, oś Y – rok). Uśrednij wartości po wszystkich stacjach w danej miejscowości. Każdy panel (facet) ma odpowiadać jednej miejscowości. Dołącz interpretację obserwowanych wyników.

In [ ]:
# Korzystamy ze scalonej ramki danych final_df
print("Informacje o final_df:")
display(final_df.head())

In [ ]:
# Przekształcamy dane do formatu 'long'
df_stacked = final_df.stack(level=['City', 'Station_Code'])
df_stacked.name = 'PM25_Value'

city_monthly_avg = df_stacked.groupby(
    [
        df_stacked.index.get_level_values('City'),
        df_stacked.index.get_level_values('Measurment').year,
        df_stacked.index.get_level_values('Measurment').month
    ]
).mean()

city_monthly_avg.index.names = ['City', 'Year', 'Month']
city_monthly_avg = city_monthly_avg.reset_index()

print("Średnie miesięczne dla miast:")
display(city_monthly_avg.head())

In [ ]:
import seaborn as sns

# Tworzymy FacetGrid, gdzie każdy panel to jedno miasto
g = sns.FacetGrid(city_monthly_avg, col="City", col_wrap=4, height=4)

# Mapujemy heatmapę na każdy panel
g.map_dataframe(
    lambda data, color: sns.heatmap(
        data.pivot(index='Year', columns='Month', values='PM25_Value'),
        cmap='viridis'
    )
)

g.set_titles("{col_name}", size=14)
g.set_axis_labels("Miesiąc", "Rok")

# Dynamiczny tytuł wykorzystujący zmienną YEARS
years_str = ", ".join(map(str, sorted(YEARS)))
g.figure.suptitle(f'Heatmapy średnich miesięcznych stężeń PM2.5 dla miast ({years_str})', size=20, y=1.03)

plt.show()

## Interpretacja zadania 3-ego

Z tego wykresu jasno widać, że ogólna sytuacja z zanieczyszceniem PM2.5 ulega poprawieniu - widać to min. w Warszawie, Katowicach czy Szczecinie. Można zaobserwować również podobne trendy do tych wyłaniających się z poprzedniego zadania. Dla większości miast stężenia cząstek PM2.5 są najwyższe od września do kwietnia. Zmniejszają się one również w kolejnych rozpatrywanych latach (2015, 2018, 2021, 2024). Gdańsk utrzymuje bardzo niskie stężenia na przestrzeni zarówno sezonów jak i lat. Może to wynikać np z łagodniejszych zim. W Kaliszu brakuje danych dla stycznie 2018. Kościerzyna w marcu 2018 zanotowała najwyższe stężenie cząstek PM2.5 w miesiącu dla przedstawionych danych.

# Zad 4
## Dni z przekroczeniem normy (WHO)
Dla każdej stacji i roku policz liczbę dni, w których wystąpiło przekroczenie dobowej normy stężenia PM2.5, czyli 15 µg/m³ (źródło: [https://airscan.org/new-who-air-quality-guidelines-2021/](https://airscan.org/new-who-air-quality-guidelines-2021/)). Znajdź 3 stacje z najmniejszą i 3 stacje z największą liczbą dni z przekroczeniem normy dobowej w 2024 roku. Dla tych 6 stacji narysuj *grouped barplot*, gdzie oś X – stacje, oś Y – liczba dni z przekroczeniem, kolor – rok (2015, 2018, 2021, 2024). Dołącz opis i interpretację obserwowanych różnic.

In [ ]:
# Norma dobowa WHO dla PM2.5
WHO_NORM = 15

daily_means_df = final_df.resample('D').mean()
exceedances = daily_means_df > WHO_NORM
exceeding_counts = exceedances.groupby(exceedances.index.year).sum()

# Filtrujemy tylko lata z konfiguracji YEARS, aby uniknąć włączenia niepełnych lat (np. 2016)
exceeding_counts = exceeding_counts[exceeding_counts.index.isin(YEARS)]

# Transponujemy, aby uzyskać stacje w wierszach i lata w kolumnach
exceeding_df = exceeding_counts.T
exceeding_df = exceeding_df.reset_index()

print("Liczba dni z przekroczeniem normy:")
display(exceeding_df.head())

In [ ]:
# Wybieramy 3 stacje z najmniejszą i 3 z największą liczbą dni z przekroczeniem w 2024
exceeding_2024 = exceeding_df.sort_values(by=2024)

# 3 stacje z najmniejszą liczbą dni
bottom_3_stations = exceeding_2024.head(3)
# 3 stacje z największą liczbą dni
top_3_stations = exceeding_2024.tail(3)

selected_stations = pd.concat([bottom_3_stations, top_3_stations])
selected_station_codes = selected_stations['Station_Code'].tolist()

print("3 stacje z najmniejszą liczbą dni z przekroczeniem normy w 2024:")
display(bottom_3_stations)
print("\n3 stacje z największą liczbą dni z przekroczeniem normy w 2024:")
display(top_3_stations)

In [ ]:
plot_data = exceeding_df[exceeding_df['Station_Code'].isin(selected_station_codes)]

# Przekształcamy dane do formatu 'long'
plot_data_long = plot_data.melt(
    id_vars=['City', 'Station_Code'],
    value_vars=YEARS, 
    var_name='Year',
    value_name='Exceeding_Days'
)

# Tworzymy etykiety dla osi X łącząc miasto i kod stacji
plot_data_long['Station_Label'] = plot_data_long['City'] + '\n(' + plot_data_long['Station_Code'] + ')'

# Rysowanie wykresu
plt.figure(figsize=(15, 8))
sns.barplot(
    data=plot_data_long,
    x='Station_Label',
    y='Exceeding_Days',
    hue='Year',
    palette='viridis'
)

plt.title('Liczba dni z przekroczeniem normy PM2.5 (15 µg/m³) dla wybranych stacji')
plt.xlabel('Stacja pomiarowa')
plt.ylabel('Liczba dni z przekroczeniem normy')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Rok')
plt.grid(axis='y')

plt.tight_layout()
plt.show()

## Interpretacja zadania 4-ego 

W pięciu z sześciu przedstawionych miast sytuacja uległa poprawie. Przerażające wydaje się, że do 2018 w Krakowie praktycznie każdego dnia były przekraczane normy (niedużo poniżej 350 dni w roku). Natomiast w 2024 sytuacji była już prawie dwukrotnie lepsza. We wszystkich latach Szczecin wydaje się być miastem w którym jakość powietrze znajduje się w ścisłej czołówce.